In [1]:
code=r"""/* ============================================================= */
/* Extended SAS script for Customer Segmentation & Enrichment    */
/* ============================================================= */

/* Global macro variable for threshold */
%let seg_threshold = 1000;

/* Step 1: Import and clean customer data */
data customers_clean;
    set raw.customer_data;
    where status = 'ACTIVE';
    total_spend = sum(spend_cat1, spend_cat2, spend_cat3);
    if total_spend < 0 then total_spend = 0;
    if missing(region) then region = 'UNKNOWN';
    format join_date date9.;
run;

/* Step 1a: Import transaction data */
data transactions_clean;
    set raw.transaction_data;
    where txn_status = 'SUCCESS';
    txn_month = month(txn_date);
    txn_year  = year(txn_date);
    if amount < 0 then amount = 0;
run;

/* Step 1b: Import loyalty program data */
data loyalty_clean;
    set raw.loyalty_data;
    where enrolled = 1;
    if missing(loyalty_points) then loyalty_points = 0;
run;

/* Step 2: Merge customer and transaction data */
proc sql;
    create table customer_txn_summary as
    select 
        a.customer_id,
        a.region,
        a.total_spend,
        count(distinct b.txn_id) as txn_count,
        sum(b.amount) as txn_total,
        avg(b.amount) as avg_txn_amount
    from customers_clean a
    left join transactions_clean b
        on a.customer_id = b.customer_id
    group by a.customer_id, a.region, a.total_spend;
quit;

/* Step 2a: Merge with loyalty data */
proc sql;
    create table customer_full_profile as
    select 
        c.*,
        l.loyalty_points,
        case 
            when l.loyalty_points >= 5000 then 'GOLD'
            when l.loyalty_points >= 1000 then 'SILVER'
            else 'BRONZE'
        end as loyalty_tier
    from customer_txn_summary c
    left join loyalty_clean l
        on c.customer_id = l.customer_id;
quit;

/* Step 3: Create customer segments */
proc sql;
    create table customer_segments as
    select 
        customer_id,
        region,
        total_spend,
        txn_total,
        loyalty_tier,
        case 
            when total_spend > &seg_threshold then 'HIGH'
            when total_spend > (&seg_threshold/2) then 'MEDIUM'
            else 'LOW'
        end as spend_segment,
        count(*) as segment_size
    from customer_full_profile
    group by calculated spend_segment, region, loyalty_tier
    order by region, loyalty_tier, calculated spend_segment;
quit;

/* Step 4: Create summary report */
proc summary data=customer_segments nway;
    class region spend_segment loyalty_tier;
    var total_spend txn_total;
    output out=segment_summary
           sum(total_spend txn_total)=total_segment_revenue total_txn_revenue
           mean(total_spend txn_total)=avg_customer_spend avg_txn_value;
run;

/* Step 5: Define report formatting macro */
%macro format_report(input_ds, output_ds, title_text);
    data &output_ds;
        set &input_ds;
        report_title = "&title_text";
        report_date = today();
        format report_date date9.;
        if total_segment_revenue > 100000 then highlight = 'YES';
        else highlight = 'NO';
    run;

    proc print data=&output_ds;
        title "&title_text";
        var region spend_segment loyalty_tier 
            total_segment_revenue total_txn_revenue 
            avg_customer_spend avg_txn_value highlight;
    run;
%mend;

/* Step 6: Generate formatted report */
%format_report(segment_summary, final_report, Customer Segment Analysis with Loyalty & Transactions);

/* Step 7: Export results */
proc export data=final_report
    outfile="segment_analysis_extended.xlsx"
    dbms=xlsx
    replace;
run;"""

In [2]:
import time
from sas_parser import parse_sas_code

start_time = time.time()

def is_container_field(key, value):
    """Check if a field contains nested SAS constructs"""
    if not isinstance(value, (list, dict)):
        return False

    # Skip metadata fields
    skip_fields = {'location', 'comments', 'errors', 'options', 'arguments'}
    if key in skip_fields:
        return False

    # Check if it's a list of objects with 'type' field
    if isinstance(value, list):
        return any(isinstance(item, dict) and 'type' in item for item in value)

    # Check if it's a dict with 'type' field or contains nested structures
    if isinstance(value, dict):
        if 'type' in value:
            return True
        # Check if any values in the dict are lists of constructs
        return any(is_container_field(k, v) for k, v in value.items())

    return False

def analyze_structure(items, indent=0, discovered_containers=None):
    if discovered_containers is None:
        discovered_containers = set()

    # Handle both list and single item
    if not isinstance(items, list):
        items = [items] if items else []

    for item in items:
        if not isinstance(item, dict):
            continue

        item_type = item.get('type', 'Unknown')
        name = item.get('macro_name', item.get('name', item.get('procedure_name', 'unnamed')))

        prefix = "  " * indent
        print(f'{prefix}• {item_type}: {name}')

        # Dynamically discover container fields
        for key, value in item.items():
            if is_container_field(key, value):
                discovered_containers.add(key)

                if isinstance(value, list) and value:
                    print(f'{prefix}  └─ {key} ({len(value)} items):')
                    analyze_structure(value, indent + 2, discovered_containers)
                elif isinstance(value, dict):
                    print(f'{prefix}  └─ {key} (dict):')
                    analyze_structure([value], indent + 2, discovered_containers)

    return discovered_containers

try:
    result = parse_sas_code(code, "nested_test.sas")
    end_time = time.time()

    print(f'✅ Parse completed in {end_time - start_time:.4f} seconds')
    print(f'📊 Body items: {len(result.get("body", []))}')

    print("\n🌳 Parsed Structure:")
    discovered_containers = analyze_structure(result.get('body', []))

    print(f"\n🔍 Discovered Container Fields:")
    for container in sorted(discovered_containers):
        print(f"   • {container}")

    # Look for any parsing issues in the debug output above
    print(f"\n🔍 Check the debug output above for:")
    print(f"   • Context push/pop messages")
    print(f"   • Any WARNING or ERROR messages")
    print(f"   • Proper nesting validation")


except Exception as e:
    end_time = time.time()
    print(f'❌ ERROR after {end_time - start_time:.4f} seconds: {e}')
    import traceback
    traceback.print_exc()

print("="*70)


[DEBUG] parse_module started at pos 0/465
[DEBUG] Parsing LET statement at pos 0
[DEBUG] Parsing DATA step at pos 5
[DEBUG] parse_unknown_statement at pos 13, token: where
[DEBUG] parse_unknown_statement completed, advanced to pos 18
[DEBUG] parse_unknown_statement at pos 48, token: format
[DEBUG] parse_unknown_statement completed, advanced to pos 53
[DEBUG] Parsing DATA step at pos 55
[DEBUG] parse_unknown_statement at pos 63, token: where
[DEBUG] parse_unknown_statement completed, advanced to pos 68
[DEBUG] Parsing DATA step at pos 93
[DEBUG] parse_unknown_statement at pos 101, token: where
[DEBUG] parse_unknown_statement completed, advanced to pos 106
[DEBUG] Parsing PROC step at pos 118
[DEBUG] Starting PROC sql statement parsing at pos 121
[DEBUG] Calling parse_proc_sql_statement for SQL at pos 121
[DEBUG] SQL statement parsed successfully, now at pos 194
[DEBUG] Parsing PROC step at pos 196
[DEBUG] Starting PROC sql statement parsing at pos 199
[DEBUG] Calling parse_proc_sql_stat

In [3]:
result

{'id': 'node_000001',
 'level': 0,
 'location': {'file': 'nested_test.sas',
  'line_start': 1,
  'col_start': 0,
  'line_end': 122,
  'col_end': 4,
  'byte_start': 0,
  'byte_end': 3676},
 'comments': ['/* ============================================================= */',
  '/* Extended SAS script for Customer Segmentation & Enrichment    */',
  '/* ============================================================= */',
  '/* Global macro variable for threshold */',
  '/* Step 1: Import and clean customer data */',
  '/* Step 1a: Import transaction data */',
  '/* Step 1b: Import loyalty program data */',
  '/* Step 2: Merge customer and transaction data */',
  '/* Step 2a: Merge with loyalty data */',
  '/* Step 3: Create customer segments */',
  '/* Step 4: Create summary report */',
  '/* Step 5: Define report formatting macro */',
  '/* Step 6: Generate formatted report */',
  '/* Step 7: Export results */'],
 'type': 'SasModule',
 'encoding': None,
 'includes': [],
 'options_snapshot':

In [4]:
from sas_ast_walker import extract_datasets_from_ast

results= extract_datasets_from_ast(result)

In [8]:
results["dataset_details"]

[DatasetInfo(name='customers_clean', type='derived', operation='data_step', location={'file': 'nested_test.sas', 'line_start': 9, 'col_start': 0, 'line_end': 16, 'col_end': 4, 'byte_start': 320, 'byte_end': 583}, node_id='node_000011'),
 DatasetInfo(name='raw.customer_data', type='source', operation='set_statement', location={'file': 'nested_test.sas', 'line_start': 10, 'col_start': 4, 'line_end': 10, 'col_end': 26, 'byte_start': 346, 'byte_end': 368}, node_id='node_000003'),
 DatasetInfo(name='transactions_clean', type='derived', operation='data_step', location={'file': 'nested_test.sas', 'line_start': 19, 'col_start': 0, 'line_end': 25, 'col_end': 4, 'byte_start': 624, 'byte_end': 817}, node_id='node_000018'),
 DatasetInfo(name='raw.transaction_data', type='source', operation='set_statement', location={'file': 'nested_test.sas', 'line_start': 20, 'col_start': 4, 'line_end': 20, 'col_end': 29, 'byte_start': 653, 'byte_end': 678}, node_id='node_000012'),
 DatasetInfo(name='loyalty_clea

In [9]:
from enhanced_ast_walker import extract_comprehensive_context

info=extract_comprehensive_context(result)

In [10]:
info

LineageExtractionContext(data_steps=[DataStepContext(step_id='node_000011', step_name='customers_clean', input_datasets=['raw.customer_data'], output_datasets=['customers_clean'], transformations=[TransformationDetail(transformation_type='filter', description="wherestatus='ACTIVE';", variables_involved=['wherestatus', 'active', ';'], logic="wherestatus='ACTIVE';", location={'file': 'nested_test.sas', 'line_start': 11, 'col_start': 4, 'line_end': 11, 'col_end': 28, 'byte_start': 373, 'byte_end': 397}), TransformationDetail(transformation_type='assignment', description='total_spend = sum ( spend_cat1 , spend_cat2 , spend_cat3 )', variables_involved=['total_spend'], logic='total_spend = sum ( spend_cat1 , spend_cat2 , spend_cat3 )', location={'file': 'nested_test.sas', 'line_start': 12, 'col_start': 4, 'line_end': 12, 'col_end': 58, 'byte_start': 402, 'byte_end': 456}), TransformationDetail(transformation_type='conditional', description='if total_spend < 0 then total_spend = 0', variables

In [11]:
from llm_lineage_generator import generate_lineage_with_llm

lin=generate_lineage_with_llm(info)

In [12]:
lin

DatasetLineage(nodes=[LineageNode(dataset_name='raw.customer_data', dataset_type='source', created_by='Source system extract', inputs=[], transformations=[], variables_created=[], business_logic='Raw customer master data containing customer demographics, status, and spending categories.'), LineageNode(dataset_name='customers_clean', dataset_type='intermediate', created_by='DATA step - customers_clean', inputs=['raw.customer_data'], transformations=["Filter records where status = 'ACTIVE'", 'Calculate total_spend = sum(spend_cat1, spend_cat2, spend_cat3)', 'If total_spend < 0 then set to 0', "If region is missing then set to 'UNKNOWN'"], variables_created=['total_spend', 'region (default UNKNOWN)'], business_logic='Active customers with cleaned and standardized spend and region information.'), LineageNode(dataset_name='raw.transaction_data', dataset_type='source', created_by='Source system extract', inputs=[], transformations=[], variables_created=[], business_logic='Raw transaction rec

In [17]:
def print_lineage_analysis(lineage):
    """Print detailed lineage analysis"""

    print("=" * 80)
    print("📊 COMPREHENSIVE DATASET LINEAGE ANALYSIS")
    print("=" * 80)

    print(f"\n🔄 DATA FLOW OVERVIEW:")
    print(f"   {lineage.flow_description}")

    # Group nodes by type
    source_nodes = [n for n in lineage.nodes if n.dataset_type == 'source']
    intermediate_nodes = [n for n in lineage.nodes if n.dataset_type == 'intermediate']
    final_nodes = [n for n in lineage.nodes if n.dataset_type == 'final']

    print(f"\n📁 DATASET INVENTORY:")
    print(f"   📥 Sources: {len(source_nodes)}")
    print(f"   ⚙️  Intermediate: {len(intermediate_nodes)}")
    print(f"   🎯 Final: {len(final_nodes)}")

    print(f"\n🗂️  SOURCE DATASETS:")
    print("-" * 50)
    for node in source_nodes:
        print(f"  📥 {node.dataset_name}")
        print(f"     Purpose: {node.business_logic}")

    print(f"\n⚙️  INTERMEDIATE DATASETS:")
    print("-" * 50)
    for node in intermediate_nodes:
        print(f"  🔄 {node.dataset_name}")
        print(f"     Created by: {node.created_by}")
        print(f"     Input datasets: {', '.join(node.inputs)}")
        print(f"     New variables: {', '.join(node.variables_created) if node.variables_created else 'None'}")
        print(f"     Business purpose: {node.business_logic}")
        if node.transformations:
            print(f"     Key transformations:")
            for transform in node.transformations[:2]:  # Show first 2
                print(f"       • {transform}")
        print()

    print(f"🔗 DATA FLOW RELATIONSHIPS:")
    print("-" * 50)
    for rel in lineage.relationships:
        print(f"  {rel.source_dataset} ➜ {rel.target_dataset}")
        print(f"    Type: {rel.transformation_type}")
        print(f"    Process: {rel.transformation_description}")
        if rel.variables_passed:
            print(f"    Variables involved: {', '.join(rel.variables_passed[:3])}...")
        print()

    print(f"🧬 VARIABLE LINEAGE:")
    print("-" * 50)
    for var, sources in lineage.variable_lineage.items():
        print(f"  {var} ← derived from: {', '.join(sources)}")

    print(f"\n📈 BUSINESS INSIGHTS:")
    print("-" * 50)
    print("  🔍 Data Quality Steps:")
    print("    • Active customer filtering (status = 'ACTIVE')")
    print("    • Successful transaction filtering (txn_status = 'SUCCESS')")
    print("    • Negative value handling (total_spend < 0 → 0)")

    print("  🔗 Integration Points:")
    print("    • Customer-Transaction join on customer_id")
    print("    • Left join preserves all customers even without transactions")

    print("  📊 Key Business Metrics:")
    print("    • total_spend: Sum of all spending categories per customer")
    print("    • txn_count: Number of distinct transactions per customer")
    print("    • txn_total: Total transaction amount per customer")
    print("    • avg_txn_amount: Average transaction size per customer")


In [18]:
print_lineage_analysis(lin)

📊 COMPREHENSIVE DATASET LINEAGE ANALYSIS

🔄 DATA FLOW OVERVIEW:
   The data flow begins with three raw source datasets: customer_data, transaction_data, and loyalty_data. Each undergoes a cleaning step to filter relevant records and standardize variables, producing customers_clean, transactions_clean, and loyalty_clean. Customers_clean and transactions_clean are joined to create customer_txn_summary, which aggregates transaction metrics per customer. This summary is then joined with loyalty_clean to produce customer_full_profile, adding loyalty points and tier classification. Finally, customer_full_profile is segmented into customer_segments based on spend thresholds, region, and loyalty tier, with segment sizes calculated. Customer_segments serves as the final analytical dataset for segmentation analysis.

📁 DATASET INVENTORY:
   📥 Sources: 3
   ⚙️  Intermediate: 5
   🎯 Final: 1

🗂️  SOURCE DATASETS:
--------------------------------------------------
  📥 raw.customer_data
     Purpose

In [20]:
from data_definition_generator import generate_data_definitions
source=r"/Users/skshahid/Downloads/code-translate-workflows-v2/sas-data-def"
data_def=generate_data_definitions(source, lin)

📁 Found 3 CSV files in /Users/skshahid/Downloads/code-translate-workflows-v2/sas-data-def
🔗 Matched 3 CSV files to source datasets
📊 Analyzing Customer_Data__Acronyms_.csv...
✅ Generated definition for customer_data
📊 Analyzing Loyalty_Data__Acronyms_.csv...
✅ Generated definition for loyalty_data
📊 Analyzing Transaction_Data__Acronyms_.csv...
✅ Generated definition for transaction_data


In [21]:
data_def

DataDefinitionContext(table_definitions=[TableDefinition(table_name='customer_data', file_path='/Users/skshahid/Downloads/code-translate-workflows-v2/sas-data-def/Customer_Data__Acronyms_.csv', business_purpose='Stores raw customer master data including personal identifiers, status, regional allocation, and spending amounts across three categories. This table serves as the foundational dataset for customer analytics, segmentation, and reporting.', columns=[ColumnDefinition(column_name='CUST_ID', data_type='VARCHAR(7)', max_length=7, is_nullable=False, constraints=['PRIMARY KEY', 'UNIQUE'], description='Unique identifier assigned to each customer in the source system.', business_rules=['Must be unique across all records', "Format: 'CUST' followed by a 3-digit number"], sample_values=['CUST001', 'CUST002', 'CUST003']), ColumnDefinition(column_name='FNAME', data_type='VARCHAR(50)', max_length=6, is_nullable=False, constraints=[], description="Customer's first name as recorded in the sourc